In [46]:
import os
import glob
import pandas as pd

In [12]:
data_patient = pd.read_csv('/content/drive/MyDrive/MSK-CHORD-new/data_clinical_patient.txt', sep='\t', skiprows = 4)
print(data_patient.columns)
col_pt = ["PATIENT_ID", "OS_MONTHS"]
df_pt = data_patient[col_pt]
df_pt.head()

,PATIENT_ID,OS_MONTHS
0,P-0000012,118.454665
1,P-0000015,13.906834
2,P-0000036,115.462887
3,P-0000041,13.610944
4,P-0000066,76.635532


In [25]:
data_sample = pd.read_csv('/content/drive/MyDrive/MSK-CHORD-new/data_clinical_sample.txt', sep = '\t', skiprows = 4)
print(data_sample.columns)
col_samp = ["SAMPLE_ID", "PATIENT_ID", "CANCER_TYPE"]
df_samp = data_sample[col_samp]
df_samp.head()

Index(['SAMPLE_ID', 'PATIENT_ID', 'GLEASON_SAMPLE_LEVEL', 'PDL1_POSITIVE',
       'CANCER_TYPE', 'SAMPLE_TYPE', 'SAMPLE_CLASS', 'METASTATIC_SITE',
       'PRIMARY_SITE', 'CANCER_TYPE_DETAILED', 'GENE_PANEL', 'SAMPLE_COVERAGE',
       'TUMOR_PURITY', 'ONCOTREE_CODE', 'MSI_COMMENT', 'MSI_SCORE', 'MSI_TYPE',
       'SOMATIC_STATUS', 'CLINICAL_GROUP', 'PATHOLOGICAL_GROUP',
       'CLINICAL_SUMMARY', 'ICD_O_HISTOLOGY_DESCRIPTION',
       'DIAGNOSIS_DESCRIPTION', 'TMB_NONSYNONYMOUS'],
      dtype='object')


,SAMPLE_ID,PATIENT_ID,CANCER_TYPE
0,P-0000012-T03-IM3,P-0000012,Non-Small Cell Lung Cancer
1,P-0000012-T02-IM3,P-0000012,Breast Cancer
2,P-0000015-T01-IM3,P-0000015,Breast Cancer
3,P-0000036-T01-IM3,P-0000036,Non-Small Cell Lung Cancer
4,P-0000041-T01-IM3,P-0000041,Breast Cancer


In [20]:
data_mut = pd.read_csv('/content/drive/MyDrive/MSK-CHORD-new/data_mutations.txt', sep ='\t')
print(data_mut.columns)
col_mut = ["Tumor_Sample_Barcode", "Patient_Identifier", "Hugo_Symbol", "Consequence", "Variant_Classification", "Variant_Type", "t_ref_count", "t_alt_count", "HGVSp_Short"]
df_mut = data_mut[col_mut]
df_mut.head()


,Tumor_Sample_Barcode,Patient_Identifier,Hugo_Symbol,Consequence,Variant_Classification,Variant_Type,t_ref_count,t_alt_count,HGVSp_Short
0,P-0081657-T01-IM7,P-0081657,EGFR,inframe_deletion,In_Frame_Del,DEL,319,288,p.L747_P753delinsS
1,P-0081657-T01-IM7,P-0081657,PDGFRB,missense_variant,Missense_Mutation,SNP,370,85,p.H271R
2,P-0081657-T01-IM7,P-0081657,RBM10,frameshift_variant,Frame_Shift_Del,DEL,189,150,p.Y519Wfs*96
3,P-0083825-T01-IM7,P-0083825,TP53,missense_variant,Missense_Mutation,SNP,187,111,p.Y205C
4,P-0083825-T01-IM7,P-0083825,TP53,stop_gained,Nonsense_Mutation,SNP,222,125,p.E294*


In [29]:
temp = pd.merge(df_pt, df_samp, on = "PATIENT_ID")
merged_df = pd.merge(temp, df_mut, left_on = "SAMPLE_ID", right_on = "Tumor_Sample_Barcode")
merged_df = merged_df.drop(["Patient_Identifier", "Tumor_Sample_Barcode"], axis = 1)
col_order = ["PATIENT_ID", "SAMPLE_ID", "CANCER_TYPE", "OS_MONTHS", "Hugo_Symbol", "Consequence", "Variant_Classification", "Variant_Type", "t_ref_count", "t_alt_count", "HGVSp_Short"]
merged_df = merged_df[col_order]
merged_df.head()

,PATIENT_ID,SAMPLE_ID,CANCER_TYPE,OS_MONTHS,Hugo_Symbol,Consequence,Variant_Classification,Variant_Type,t_ref_count,t_alt_count,HGVSp_Short
0,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,PIK3R3,missense_variant,Missense_Mutation,SNP,604,129,p.I298M
1,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,ATR,missense_variant,Missense_Mutation,SNP,398,84,p.R2431M
2,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,PDGFRA,missense_variant,Missense_Mutation,SNP,454,116,p.L465M
3,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,FBXW7,splice_acceptor_variant,Splice_Site,SNP,248,85,p.X413_splice
4,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,FBXW7,stop_gained,Nonsense_Mutation,SNP,440,130,p.G61*


In [30]:
#calculate VAF
merged_df["VAF"] = merged_df["t_alt_count"] / (merged_df["t_ref_count"] + merged_df["t_alt_count"])
merged_df.head()

,PATIENT_ID,SAMPLE_ID,CANCER_TYPE,OS_MONTHS,Hugo_Symbol,Consequence,Variant_Classification,Variant_Type,t_ref_count,t_alt_count,HGVSp_Short,VAF
0,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,PIK3R3,missense_variant,Missense_Mutation,SNP,604,129,p.I298M,0.175989
1,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,ATR,missense_variant,Missense_Mutation,SNP,398,84,p.R2431M,0.174274
2,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,PDGFRA,missense_variant,Missense_Mutation,SNP,454,116,p.L465M,0.203509
3,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,FBXW7,splice_acceptor_variant,Splice_Site,SNP,248,85,p.X413_splice,0.255255
4,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,FBXW7,stop_gained,Nonsense_Mutation,SNP,440,130,p.G61*,0.228070


In [33]:
#write out individual cancer files
cancer_types = merged_df["CANCER_TYPE"].unique()
for cancer in cancer_types:
  cancer_df = merged_df[merged_df["CANCER_TYPE"] == cancer]
  cancer_df.to_csv(f"/content/drive/MyDrive/MSK-CHORD-new/cancer_files/{cancer}.csv", index = False)

In [45]:
pan = pd.read_csv('/content/drive/MyDrive/MSK-CHORD-new/cancer_files/Pancreatic Cancer.csv')
pan = pan.drop(["CANCER_TYPE", "t_ref_count", "t_alt_count"], axis = 1)
pan["n_patients"] = pan.groupby("Hugo_Symbol")["PATIENT_ID"].transform("nunique")
total_pt = pan["PATIENT_ID"].nunique()
pan ['Mut_freq'] = pan["n_patients"] / total_pt
pan ["Gene_Score"] = pan["Mut_freq"] * pan["OS_MONTHS"]
col_reorder = ["PATIENT_ID", "SAMPLE_ID", "Hugo_Symbol","n_patients", "Mut_freq", "OS_MONTHS", "Gene_Score", "Consequence", "Variant_Classification", "Variant_Type", "HGVSp_Short", "VAF"]
pan = pan[col_reorder]
pan.head()

,PATIENT_ID,SAMPLE_ID,Hugo_Symbol,n_patients,Mut_freq,OS_MONTHS,Gene_Score,Consequence,Variant_Classification,Variant_Type,HGVSp_Short,VAF
0,P-0000142,P-0000142-T01-IM3,KRAS,2516,0.851438,8.810949,7.501979,missense_variant,Missense_Mutation,SNP,p.G12D,0.194444
1,P-0000142,P-0000142-T01-IM3,TP53,2076,0.702538,8.810949,6.190027,stop_gained,Nonsense_Mutation,SNP,p.R213*,0.078987
2,P-0000142,P-0000142-T01-IM3,MSH2,12,0.004061,8.810949,0.035781,missense_variant,Missense_Mutation,SNP,p.L421R,0.064449
3,P-0000142,P-0000142-T01-IM3,TERT,37,0.012521,8.810949,0.110323,missense_variant,Missense_Mutation,SNP,p.R671W,0.149780
4,P-0000314,P-0000314-T01-IM3,PARP1,11,0.003723,33.731470,0.125566,missense_variant,Missense_Mutation,SNP,p.T124M,0.300765


In [51]:
# 1. Define input/output directories
input_dir  = "/content/drive/MyDrive/MSK-CHORD-new/cancer_files"
output_dir = "/content/drive/MyDrive/MSK-CHORD-new/cancer_files_scored"
os.makedirs(output_dir, exist_ok=True)

# 2. Loop through each cancer file
for filepath in glob.glob(os.path.join(input_dir, "*.csv")):
    # read
    file = pd.read_csv(filepath)

    # drop unwanted columns
    file = file.drop(["SAMPLE_ID", "Consequence",
                      "Variant_Classification", "Variant_Type",
                      "HGVSp_Short", "VAF","CANCER_TYPE",
                      "t_ref_count", "t_alt_count",
                      "HGVSp_Short", "VAF"], axis=1)

    # compute number of distinct patients per gene
    file["n_patients"] = file.groupby("Hugo_Symbol")["PATIENT_ID"] \
                           .transform("nunique")

    # total patients in this cohort
    total_pt = file["PATIENT_ID"].nunique()

    # mutation frequency and gene score
    file["Mut_freq"]   = file["n_patients"]    / total_pt
    file["Gene_Score"] = file["Mut_freq"] * pan["OS_MONTHS"]

    # drop the patient column, then collapse to one row per Hugo_Symbol
    file = (file
            .drop(["PATIENT_ID"], axis=1)
            .drop_duplicates(subset=["Hugo_Symbol"], keep="first")
            )

    # reorder columns
    col_reorder = ["Hugo_Symbol","n_patients", "Mut_freq", "OS_MONTHS", "Gene_Score"]
    file = file[col_reorder]

    # write out
    basename = os.path.basename(filepath)
    name, _  = os.path.splitext(basename)
    outpath  = os.path.join(output_dir, f"{name}_scored.csv")
    file.to_csv(outpath, index=False)